# Exploring coherence in Grand Mesa

I want to explore one of the SnowEx Grand Mesa time series and look at how coherence changes over time. 

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import xarray as xr 
import matplotlib.pyplot as plt

Current working directory should be /Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence but check this and change if necessary 

In [4]:
os.getcwd()
os.chdir('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/')

Now get the data that we want. I'll look at tjust the 2020 timeseries, which has 7 image pairs 

In [16]:
data_dir = './data/data/snowex_grmesa'
# scenes = os.listdir(data_dir)

In [17]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if re.search(pattern, name):
                result.append(os.path.join(root, name))
    return result

In [18]:
find('_20.*HH.*cor', data_dir)

['./data/data/snowex_grmesa/grmesa_27416_20005-007_20008-004_0007d_s01_L090_01_int_grd/grmesa_27416_20005-007_20008-004_0007d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20003-028_20008-004_0018d_s01_L090_01_int_grd/grmesa_27416_20003-028_20008-004_0018d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20003-028_20017-006_0040d_s01_L090_01_int_grd/grmesa_27416_20003-028_20017-006_0040d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20008-004_20013-004_0007d_s01_L090_01_int_grd/grmesa_27416_20008-004_20013-004_0007d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20003-028_20013-004_0025d_s01_L090_01_int_grd/grmesa_27416_20003-028_20013-004_0025d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20003-028_20005-007_0011d_s01_L090_01_int_grd/grmesa_27416_20003-028_20005-007_0011d_s01_L090HH_01.cor.grd.tiff',
 './data/data/snowex_grmesa/grmesa_27416_20013-004_20017-006_0015d_s01

We have the filenames for the coherence files of all HH polarizations in 2020. For simplicity, I will just use the HH polarizations. 